In [1]:
!vagrant status

Current machine states:

k8s-master                not created (virtualbox)
k8s-worker1               not created (virtualbox)
k8s-worker2               not created (virtualbox)
k8s-client                not created (virtualbox)

This environment represents multiple VMs. The VMs are all listed
above with their current state. For more information about a specific
VM, run `vagrant status NAME`.


In [2]:
!vagrant box list

ubuntu/focal64 (virtualbox, 20240821.0.1)


In [3]:
!vagrant status

Current machine states:

k8s-master                running (virtualbox)
k8s-worker1               running (virtualbox)
k8s-worker2               running (virtualbox)
k8s-client                running (virtualbox)

This environment represents multiple VMs. The VMs are all listed
above with their current state. For more information about a specific
VM, run `vagrant status NAME`.


In [4]:
!vagrant ssh k8s-client -c "kubectl get nodes -o wide"

NAME          STATUS   ROLES           AGE     VERSION    INTERNAL-IP   EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION      CONTAINER-RUNTIME
k8s-master    Ready    control-plane   4m39s   v1.30.14   10.0.2.15     <none>        Ubuntu 20.04.6 LTS   5.4.0-216-generic   containerd://1.7.27
k8s-worker1   Ready    <none>          2m59s   v1.30.14   10.0.2.15     <none>        Ubuntu 20.04.6 LTS   5.4.0-216-generic   containerd://1.7.27
k8s-worker2   Ready    <none>          104s    v1.30.14   10.0.2.15     <none>        Ubuntu 20.04.6 LTS   5.4.0-216-generic   containerd://1.7.27


In [5]:
!vagrant ssh k8s-client -c"kubectl apply -f https://raw.githubusercontent.com/kubernetes/dashboard/v2.7.0/aio/deploy/recommended.yaml"

namespace/kubernetes-dashboard created
serviceaccount/kubernetes-dashboard created
service/kubernetes-dashboard created
secret/kubernetes-dashboard-certs created
secret/kubernetes-dashboard-csrf created
secret/kubernetes-dashboard-key-holder created
configmap/kubernetes-dashboard-settings created
role.rbac.authorization.k8s.io/kubernetes-dashboard created
clusterrole.rbac.authorization.k8s.io/kubernetes-dashboard created
rolebinding.rbac.authorization.k8s.io/kubernetes-dashboard created
clusterrolebinding.rbac.authorization.k8s.io/kubernetes-dashboard created
deployment.apps/kubernetes-dashboard created
service/dashboard-metrics-scraper created
deployment.apps/dashboard-metrics-scraper created


dashboard-adminuser.yaml

apiVersion: v1
kind: ServiceAccount
metadata:
  name: admin-user
  namespace: kubernetes-dashboard
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: admin-user
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: cluster-admin
subjects:
- kind: ServiceAccount
  name: admin-user
  namespace: kubernetes-dashboard

!vagrant ssh k8s-client -c"kubectl apply -f dashboard-adminuser.yaml"

In [6]:
!vagrant ssh k8s-client -c"kubectl -n kubernetes-dashboard create token admin-user"

eyJhbGciOiJSUzI1NiIsImtpZCI6IlhZZFQtZmhZVXNIcDJOd2o2YzQ4R0ZEM09xT0tfOTRweGsyZlNrdUl4dXcifQ.eyJhdWQiOlsiaHR0cHM6Ly9rdWJlcm5ldGVzLmRlZmF1bHQuc3ZjLmNsdXN0ZXIubG9jYWwiXSwiZXhwIjoxNzU1OTk5MTQ2LCJpYXQiOjE3NTU5OTU1NDYsImlzcyI6Imh0dHBzOi8va3ViZXJuZXRlcy5kZWZhdWx0LnN2Yy5jbHVzdGVyLmxvY2FsIiwianRpIjoiYmM0NjZlZjEtYjgxYi00MzIzLTg0NDEtMTU1ODEwMDc2ZmI2Iiwia3ViZXJuZXRlcy5pbyI6eyJuYW1lc3BhY2UiOiJrdWJlcm5ldGVzLWRhc2hib2FyZCIsInNlcnZpY2VhY2NvdW50Ijp7Im5hbWUiOiJhZG1pbi11c2VyIiwidWlkIjoiODBjYzQ3YzctMTJiMS00NzFkLTg5ZTctOTMzYWM4YjE3NWM1In19LCJuYmYiOjE3NTU5OTU1NDYsInN1YiI6InN5c3RlbTpzZXJ2aWNlYWNjb3VudDprdWJlcm5ldGVzLWRhc2hib2FyZDphZG1pbi11c2VyIn0.TOQMW32XPJt07w6EF1SsNfPaW-6MRCuPOPDbU0DrjSJX7SgmDfsdHZMU1Uf-piNVq46yrpo1hZLmNzOvacc8eGOtpL85xhx3os-We5VT-3CGV6FHmeEc2Vc7MHY0usJt3YRL-FyrEPndc3RWDX4rSQz2VbKTs5CzKZdormLqbUgCBQ8rMaBhEp7V-IX3U-Hwa2PIhv4tJzdknAwZE7EEJ4-tfEVZhLxYcx_LUcky5i63M3yZsZyeuc9K0zGV97UW0HKBiFz9zekbG5bK_MuMp0oElryyNW2BdAjY38GhcYJvd_EveL7I4UX2QBFJCsxXfDQD5b2a-Do9xmS-X1RlAA


In [ ]:
!vagrant ssh k8s-client -c"kubectl proxy --address=0.0.0.0 --disable-filter=true &"

In [7]:
!vagrant ssh k8s-client -c"hostname -I"

10.0.2.15 192.168.56.13 fd17:625c:f037:2:1a:a2ff:fe1e:9db7 


http://192.168.56.13:8001/api/v1/namespaces/kubernetes-dashboard/services/https:kubernetes-dashboard:/proxy/#/login

In [9]:
!vagrant ssh k8s-client -c"kubectl create deployment nginx --image=nginx:latest --replicas=2"

error: failed to create deployment: deployments.apps "nginx" already exists


In [10]:
!vagrant ssh k8s-client -c"kubectl get pods"

NAME                     READY   STATUS    RESTARTS   AGE
nginx-7584b6f84c-29dk4   1/1     Running   0          60s
nginx-7584b6f84c-lxcvt   1/1     Running   0          60s


In [11]:
!vagrant ssh k8s-client -c"kubectl scale deployment nginx --replicas=10"

deployment.apps/nginx scaled


In [12]:
!vagrant ssh k8s-client -c"kubectl get deployments"

NAME    READY   UP-TO-DATE   AVAILABLE   AGE
nginx   10/10   10           10          113s


In [14]:
!vagrant ssh k8s-client -c"kubectl get pods -o wide"

NAME                     READY   STATUS    RESTARTS   AGE     IP               NODE          NOMINATED NODE   READINESS GATES
nginx-7584b6f84c-29dk4   1/1     Running   0          2m25s   192.168.194.66   k8s-worker1   <none>           <none>
nginx-7584b6f84c-4ftjv   1/1     Running   0          50s     192.168.194.68   k8s-worker1   <none>           <none>
nginx-7584b6f84c-7fsjx   1/1     Running   0          50s     192.168.194.70   k8s-worker1   <none>           <none>
nginx-7584b6f84c-8h6rf   1/1     Running   0          50s     192.168.126.5    k8s-worker2   <none>           <none>
nginx-7584b6f84c-9j6qr   1/1     Running   0          50s     192.168.126.3    k8s-worker2   <none>           <none>
nginx-7584b6f84c-jszg2   1/1     Running   0          50s     192.168.126.4    k8s-worker2   <none>           <none>
nginx-7584b6f84c-lm6vv   1/1     Running   0          50s     192.168.194.67   k8s-worker1   <none>           <none>
nginx-7584b6f84c-lxcvt   1/1     Running   0          2